## Import libraries

In [1]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap
from fiona.crs import from_epsg
import matplotlib as mpl
import matplotlib.pyplot as plt
import bokeh
from bokeh import plotting
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.tile_providers import CARTODBPOSITRON
import psycopg2

NoneType = type(None)

%matplotlib inline

## Data addresses

1. **Stations request** http://api.gios.gov.pl/pjp-api/rest/station/findAll
2. **Sensors request** http://api.gios.gov.pl/pjp-api/rest/station/sensors/{stationId}
3. **Data request** http://api.gios.gov.pl/pjp-api/rest/data/getData/{sensorId}
4. **AQ index request** http://api.gios.gov.pl/pjp-api/rest/aqindex/getIndex/{stationId}

# Geopandas part

## Get Stations from API

In [2]:
stations = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/findAll").json()
print("Number of available stations: ", len(stations))

Number of available stations:  157


In [3]:
stations_dict = {}
station_ids = []
station_lats = []
station_lons = []
station_geometries = []

for station in stations:
    
    station_ids.append(station["id"])
    #station_lats.append(float(station["gegrLat"]))
    #station_lons.append(float(station["gegrLon"]))
    station_geometries.append(Point(float(station["gegrLon"]), float(station["gegrLat"])))
    
stations_dict["station_id"] = station_ids
#stations_dict["lat"] = station_lats
#stations_dict["lon"] = station_lons
stations_dict["station_id"] = station_ids
stations_dict["geometry"] = station_geometries

In [4]:
stations_df = gpd.GeoDataFrame(stations_dict)
stations_df.crs = from_epsg(4326)
stations_df.head()

,station_id,geometry
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## Plot Stations on map

In [5]:
stations_map = folium.Map([52, 19], zoom_start=6, tiles='Stamen Terrain')

points = folium.features.GeoJson(stations_df.to_json())

stations_map.add_child(points)
#stations_map.add_child(HeatMap([[row["lat"], row["lon"]] for name, row in stations_df.iterrows()]))
stations_map

## Get list of Sensors for each Station

In [6]:
sensors_dict = {}
stations_ids = []
sensors_ids = []
sensors_param = []

for station in stations:
    
    station_id = station["id"]
    sensors = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    for sensor in sensors:

        stations_ids.append(sensor["stationId"])
        sensors_ids.append(sensor["id"])
        sensors_param.append(sensor["param"]["paramCode"])
    
sensors_dict["station_id"] = stations_ids
sensors_dict["sensor_id"] = sensors_ids
sensors_dict["parameter"] = sensors_param\

In [9]:
sensors_df = pd.DataFrame(sensors_dict)
sensors_df.head()

,station_id,sensor_id,parameter
0,114,642,NO2
1,114,644,O3
2,117,660,CO
3,117,14395,PM10
4,117,658,C6H6


## Get Sensors readings from API

In [10]:
def get_sensor_readings(row):
    
    sensor_id = row["sensor_id"]
    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    count = 0
    try:
        data = data_json["values"][count]["value"]
        while isinstance(data, NoneType):
            data = data_json["values"][count]["value"]
            count+=1
        return data
    except:
        return np.NaN

In [11]:
sensors_df['data'] = sensors_df.apply(get_sensor_readings, axis=1)
sensors_df.head()

,station_id,sensor_id,parameter,data
0,114,642,NO2,12.76580
1,114,644,O3,99.66110
2,117,660,CO,381.52700
3,117,14395,PM10,19.23480
4,117,658,C6H6,0.00035


In [12]:
available_parameters = list(sensors_df.parameter.unique())
available_parameters

['NO2', 'O3', 'CO', 'PM10', 'C6H6', 'PM2.5', 'SO2']

In [13]:
def get_param_df(parameter=''):
    
    param_df = sensors_df[sensors_df["parameter"]=="{}".format(parameter)]
    param_df = gpd.GeoDataFrame(pd.merge(param_df, stations_df, on='station_id'))
    param_df.dropna(inplace=True)
    param_df.crs = from_epsg(4326)
    
    return param_df

In [14]:
def show_readings(parameter=''):
    
    param_df = get_param_df("{}".format(parameter))
    param_df = param_df.to_crs(epsg=3395)  # conversion to World Mercator needed
    
    plotting.output_notebook()

    # reduce differences between readings
    size=param_df["data"]
    # get data resolution
    radii = np.array(param_df['data'].apply(int))
    #create colors list
    colors = ["#%02x%02x%02x" % (int(r), int(g), int(b)) for r, g, b, _ in 255*mpl.cm.RdYlGn(1-mpl.colors.Normalize()(radii))]

    p = plotting.figure(toolbar_location="left", 
                        plot_width=900, 
                        plot_height=700, 
                        x_axis_type="mercator", 
                        y_axis_type="mercator")

    p.circle(param_df['geometry'].x, 
             param_df['geometry'].y, 
             size=size, 
             fill_color=colors, 
             fill_alpha=0.8, 
             line_color=None)

    p.add_tile(CARTODBPOSITRON)

    plotting.show(p)

## Visualize readings

In [20]:
show_readings("O3")

Loading BokehJS ...

# Connect with database

In [72]:
try:
    conn = psycopg2.connect("dbname='haqs' user='postgres' host='localhost' password='postgres'")
except:
    print ("I am unable to connect to the database")
    conn.close()

In [71]:
conn.close()

## Create PostGIS Extension

In [29]:
sql = "CREATE EXTENSION postgis;"

cur = conn.cursor()
try:
    cur.execute(sql)
except psycopg2.ProgrammingError as e:
    print (e)

BŁĄD:  rozszerzenie "postgis" już istnieje



## <font color="blue">Show Tables</font>

In [ ]:
sql = "SELECT * FROM pg_catalog.pg_tables;"

cur = conn.cursor()
cur.execute(sql)
cur.fetchall()

## <font color="blue">Station Table</font>

### Create Stations Table

In [47]:
sql = """
CREATE TABLE public.stations 
( 
    station_id INTEGER PRIMARY KEY
);

SELECT AddGeometryColumn('stations', 'geom', '4326', 'POINT', 2);
"""

cur = conn.cursor()
cur.execute(sql)

### Insert Stations into DataBase

1. Using text 
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_GeomFromText('POINT(-71.060316 48.432044)', EPSG));
2. Using lognitude and latitude
> <i>**INSERT INTO**</i> stations(station_id, geom) **VALUES**(2, ST_SetSRID(ST_MakePoint(lon, lat), EPSG));

In [ ]:
stations = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/findAll").json()

In [49]:
stations[0]

{'id': 114,
 'stationName': 'Wrocław - Bartnicza',
 'gegrLat': '51.115933',
 'gegrLon': '17.141125',
 'city': {'id': 1064,
  'name': 'Wrocław',
  'commune': {'communeName': 'Wrocław',
   'districtName': 'Wrocław',
   'provinceName': 'DOLNOŚLĄSKIE'}},
 'addressStreet': 'ul. Bartnicza'}

In [70]:
def db_insert_station(conn, station_id, lon, lat):
    """
    Function inserts stations with its coordinates to Database
    """
    sql = "INSERT INTO public.stations (station_id, geom) VALUES (%s, ST_SetSRID(ST_MakePoint(%s, %s), 4326));"
    with conn.cursor() as cur:
        cur.execute(sql, (station_id, lon, lat))
    conn.commit()

In [71]:
# iterate over stations
for station in stations:
    
    station_id = station["id"]
    station_lon = station['gegrLon']
    station_lat = station['gegrLat']
    
    try:
        db_insert_station(conn=conn, station_id=station_id, lon=station_lon, lat=station_lat)
    except:
        print ("Cannot execute insertion for Station ID: {}".format(station_id))

### Show Insertions

In [ ]:
sql = "SELECT * FROM public.stations"

cur = conn.cursor()
cur.execute(sql)
cur.fetchall()

### Create Stations DataFrame from Stations Table

In [3]:
sql = "SELECT * FROM public.stations"

stations_df = gpd.read_postgis(sql, conn, geom_col='geom')

In [4]:
stations_df.head()

,station_id,geom
0,114,POINT (17.141125 51.115933)
1,117,POINT (17.02925 51.129378)
2,129,POINT (17.012689 51.086225)
3,52,POINT (16.180513 51.204503)
4,109,POINT (16.269677 50.768729)


## <font color="blue">Sensors Table</font>

### Create Sensors Table

In [89]:
sql = """
    CREATE TABLE public.sensors 
    (
        sensor_id INTEGER PRIMARY KEY,
        sensor_parameter VARCHAR(10) NOT NULL,
        station_id INTEGER REFERENCES stations (station_id)
    );
"""
    
cur = conn.cursor()
cur.execute(sql)

### Insert Sensors into DataBase

> <i>**INSERT INTO**</i> sensors(sensor_id, sensor_parameter, station_id) **VALUES** (2, 'PM10', 114));

In [137]:
def db_insert_station_sensors(conn, sensor_id, sensor_parameter, station_id):
    """
    Function inserts sensors for each station to Database
    """
    sql = """INSERT INTO public.sensors (sensor_id, sensor_parameter, station_id)
                VALUES (%s, %s, %s);"""
    
    with conn.cursor() as cur:
        cur.execute(sql, (sensor_id, sensor_parameter, station_id))
    conn.commit()

In [ ]:
# iterate over stations
for station in stations:
    
    station_id = station["id"]
    sensors = requests.get("http://api.gios.gov.pl/pjp-api/rest/station/sensors/{}".format(station_id)).json()
    
    # iterate over station sensors
    for sensor in sensors:
        
        sensor_id = sensor["id"]
        sensor_parameter = sensor["param"]["paramCode"]
        
        try:
            db_insert_station_sensors(conn, sensor_id, sensor_parameter, station_id)
        except:
            print ("Cannot execute insertion for Sensor ID: {} (Station ID: {})".format(sensor_id, station_id))

### Create Sensors DataFrame from Sensors Table

In [79]:
sql = "SELECT * FROM public.sensors"

sensors_df = pd.read_sql_query(sql,con=conn)
sensors_df.head()

,sensor_id,sensor_parameter,station_id
0,642,NO2,114
1,644,O3,114
2,660,CO,117
3,14395,PM10,117
4,658,C6H6,117


## <font color="blue">Readings Table</font>

### Create Readings Table  

In [ ]:
sql = """
    CREATE TABLE public.readings 
    (
        sensor_id INTEGER REFERENCES sensors (sensor_id),
        date VARCHAR(19) NOT NULL,
        reading FLOAT(4)
    );
"""
    
cur = conn.cursor()
cur.execute(sql)

### Create list of available Sensors

In [47]:
sql = "SELECT sensor_id FROM sensors;"

cur = conn.cursor()
cur.execute(sql)
sensors_ids = [values[0] for values in cur.fetchall()]
sensors_ids

[642,
 644,
 660,
 14395,
 658,
 665,
 670,
 667,
 672,
 737,
 740,
 744,
 285,
 14397,
 282,
 291,
 293,
 297,
 608,
 618,
 605,
 614,
 616,
 621,
 50,
 52,
 56,
 92,
 88,
 94,
 101,
 99,
 102,
 224,
 221,
 223,
 225,
 14706,
 397,
 402,
 441,
 444,
 450,
 493,
 759,
 757,
 760,
 771,
 14735,
 776,
 782,
 14707,
 14730,
 14734,
 14727,
 14731,
 14729,
 14733,
 16415,
 16412,
 16417,
 16414,
 16413,
 16422,
 16419,
 16393,
 16421,
 16420,
 952,
 965,
 949,
 959,
 966,
 969,
 987,
 995,
 991,
 996,
 993,
 998,
 2060,
 2069,
 2064,
 2071,
 2066,
 2076,
 2080,
 2087,
 2079,
 2083,
 16805,
 2085,
 2088,
 2199,
 2195,
 2197,
 2204,
 2370,
 2377,
 2373,
 2378,
 2382,
 16228,
 16250,
 16249,
 16231,
 2035,
 2031,
 2033,
 2039,
 2219,
 2221,
 2222,
 2239,
 2245,
 2242,
 2244,
 2248,
 16642,
 16630,
 16638,
 17716,
 16633,
 3802,
 14379,
 16037,
 16041,
 16043,
 17232,
 17239,
 17235,
 17240,
 17237,
 17241,
 16160,
 16158,
 16149,
 16147,
 3836,
 3843,
 3834,
 3839,
 3844,
 3841,
 3847,
 3902,

### Insert Readings Into DataBase

> <i><b>INSERT INTO**</b></i> readings(sensor_id, date, reading) <i><b>VALUES</b></i> (621, '2018-09-05 00:00:00', 32.2259));

In [74]:
def db_insert_sensor_readings(conn, sensor_id, date, reading):
    """
    Function inserts multiple readings for sensor
    """
    
    sql = """
        INSERT INTO readings (sensor_id, date, reading)
        SELECT %s, %s, %s
        WHERE
            NOT EXISTS 
            (
                SELECT * FROM readings WHERE date = %s AND sensor_id = %s
            )
    """
    
    with conn.cursor() as cur:
        cur.execute(sql, (sensor_id, date, reading, date, sensor_id))
    conn.commit()

#### Insert all available non NULL readings into Readings Table

In [78]:
for sensor_id in sensors_ids:

    data_json = requests.get("http://api.gios.gov.pl/pjp-api/rest/data/getData/{}".format(sensor_id)).json()
    data = data_json['values']
    
    # check all available data
    for row in data:

        date = row['date']
        reading = row['value']

        # if there is no value wait until next reading
        if isinstance(reading, NoneType):
            continue

        try:
            db_insert_sensor_readings(conn, sensor_id, date, reading)
        except Exception as e:
            print(e)

### Create Readings DataFrame from Readings Table

In [80]:
sql = "SELECT * FROM public.readings"

readings_df = pd.read_sql_query(sql,con=conn)
readings_df.head()

,sensor_id,date,reading
0,642,2018-09-07 13:00:00,11.3334
1,642,2018-09-07 12:00:00,12.7021
2,642,2018-09-07 11:00:00,15.3369
3,642,2018-09-07 10:00:00,16.7608
4,642,2018-09-07 09:00:00,19.4419


In [82]:
readings_df.describe()

,sensor_id,reading
count,38018.000000,38018.000000
mean,6930.134384,64.058907
std,5884.381670,135.262029
min,50.000000,0.000000
25%,2664.000000,4.575068
50%,4793.000000,18.189650
75%,14395.000000,41.380750
max,19816.000000,2928.870000


In [83]:
readings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38018 entries, 0 to 38017
Data columns (total 3 columns):
sensor_id    38018 non-null int64
date         38018 non-null object
reading      38018 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 891.1+ KB


In [87]:
readings_df[(readings_df['date']=='2018-09-07 14:00:00') & (readings_df['sensor_id']==642)]

,sensor_id,date,reading
37583,642,2018-09-07 14:00:00,10.2202
